In [3]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, models, LoggingHandler, losses, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from sentence_transformers.datasets import NoDuplicatesDataLoader

/opt/conda/envs/iris/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# SEED 설정
import random
seed = 7777
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [5]:
# LOGGER 초기화
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y/%m/%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [6]:
pretrained_model_name = 'klue/roberta-base'
nli_num_epochs = 1
sts_num_epochs = 4
train_batch_size = 32

nli_model_save_path = 'output/training_nli_by_Softmaxloss'+pretrained_model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
sts_model_save_path = 'output/training_sts_by_Softmaxloss'+pretrained_model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [7]:
# load KLUE-NLI Dataset
klue_nli_train = load_dataset("klue", "nli", split='train')
print('Length of Train : ',len(klue_nli_train))

Generating validation split: 100%|██████████| 3000/3000 [00:00<00:00, 9593.25 examples/s] 

Length of Train :  24998


In [8]:
def make_nli_input_example(dataset):
    ''' 
    Transform to InputExample
    ''' 
    input_examples = []
    for i, data in enumerate(dataset):
        sentence1 = data['hypothesis']
        sentence2 = data['premise']
        label = data['label'] # 0(entailment), 1(neutral), 2(contradiction)
        input_examples.append(InputExample(texts=[sentence1, sentence2], label=label))

    return input_examples

In [9]:
nli_train_examples = make_nli_input_example(klue_nli_train)

In [12]:
# Train Dataloader
train_dataloader = DataLoader(
    nli_train_examples,
    shuffle=True,
    batch_size=train_batch_size,
)

In [13]:
# Load Embedding Model
embedding_model = models.Transformer(
    model_name_or_path=pretrained_model_name, 
    max_seq_length=256,
    do_lower_case=True
)

# Only use Mean Pooling -> Pooling all token embedding vectors of sentence.
pooling_model = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

model = SentenceTransformer(modules=[embedding_model, pooling_model])

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2023/08/08 07:08:50 - Use pytorch device: cuda


In [14]:
# Use SoftmaxLoss, because NLI is Multi-class Classification task.
train_loss = losses.SoftmaxLoss(
    model=model, 
    sentence_embedding_dimension=model.get_sentence_embedding_dimension(), 
    num_labels=3 # entailment, neutral, contradiction
)

# warmup steps
warmup_steps = math.ceil(len(nli_train_examples) * nli_num_epochs / train_batch_size * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Training
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=dev_evaluator,
    epochs=nli_num_epochs,
    evaluation_steps=int(len(train_dataloader)*0.1),
    warmup_steps=warmup_steps,
    output_path=nli_model_save_path
)

2023/08/08 07:08:50 - Softmax loss: #Vectors concatenated: 3
2023/08/08 07:08:50 - Warmup-steps: 79


Iteration:  10%|▉         | 76/782 [00:12<01:56,  6.07it/s]
                                            :56,  6.06it/s]
Iteration:  10%|▉         | 77/782 [00:13<01:56,  6.06it/s]

2023/08/08 07:09:05 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset in epoch 0 after 78 steps:


                                            
Epoch:   0%|          | 0/1 [00:16<?, ?it/s]               
                                            :56,  6.06it/s]
Epoch:   0%|          | 0/1 [00:16<?, ?it/s]               
                                            :56,  6.06it/s]
Epoch:   0%|          | 0/1 [00:16<?, ?it/s]               
                                            :56,  6.06it/s]
Epoch:   0%|          | 0/1 [00:16<?, ?it/s]               
                                            :56,  6.06it/s]
Iteration:  10%|▉         | 77/782 [00:16<01:56,  6.06it/s]

2023/08/08 07:09:08 - Cosine-Similarity :	Pearson: 0.8382	Spearman: 0.8563
2023/08/08 07:09:08 - Manhattan-Distance:	Pearson: 0.8317	Spearman: 0.8473
2023/08/08 07:09:08 - Euclidean-Distance:	Pearson: 0.8278	Spearman: 0.8445
2023/08/08 07:09:08 - Dot-Product-Similarity:	Pearson: 0.7066	Spearman: 0.7190
2023/08/08 07:09:09 - Save model to output/training_nli_by_Softmaxlossklue-roberta-base-2023-08-08_07-08-05


...

2023/08/08 07:11:40 - Cosine-Similarity :	Pearson: 0.6064	Spearman: 0.6883
2023/08/08 07:11:40 - Manhattan-Distance:	Pearson: 0.6761	Spearman: 0.7012
2023/08/08 07:11:40 - Euclidean-Distance:	Pearson: 0.6674	Spearman: 0.7003
2023/08/08 07:11:40 - Dot-Product-Similarity:	Pearson: 0.5751	Spearman: 0.5950


Epoch:   0%|          | 0/1 [02:48<?, ?it/s]

2023/08/08 07:11:40 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 0:


Epoch: 100%|██████████| 1/1 [02:52<00:00, 172.32s/it]

2023/08/08 07:11:44 - Cosine-Similarity :	Pearson: 0.6064	Spearman: 0.6883
2023/08/08 07:11:44 - Manhattan-Distance:	Pearson: 0.6761	Spearman: 0.7012
2023/08/08 07:11:44 - Euclidean-Distance:	Pearson: 0.6674	Spearman: 0.7003
2023/08/08 07:11:44 - Dot-Product-Similarity:	Pearson: 0.5751	Spearman: 0.5950


In [15]:
# load KLUE-STS Dataset
klue_sts_train = load_dataset("klue", "sts", split='train[:90%]')
klue_sts_valid = load_dataset("klue", "sts", split='train[-10%:]') # train의 10%를 validation set으로 사용
klue_sts_test = load_dataset("klue", "sts", split='validation')

print('Length of Train : ',len(klue_sts_train))
print('Length of Valid : ',len(klue_sts_valid))
print('Length of Test : ',len(klue_sts_test))

def make_sts_input_example(dataset):
    ''' 
    Transform to InputExample
    ''' 
    input_examples = []
    for i, data in enumerate(dataset):
        sentence1 = data['sentence1']
        sentence2 = data['sentence2']
        score = (data['labels']['label']) / 5.0  # normalize 0 to 5
        input_examples.append(InputExample(texts=[sentence1, sentence2], label=score))

    return input_examples

sts_train_examples = make_sts_input_example(klue_sts_train)
sts_valid_examples = make_sts_input_example(klue_sts_valid)
sts_test_examples = make_sts_input_example(klue_sts_test)

# Train Dataloader
train_dataloader = DataLoader(
    sts_train_examples,
    shuffle=True,
    batch_size=train_batch_size,
)

# Evaluator by sts-validation
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    sts_valid_examples,
    name="sts-dev",
)

# Evaluator by sts-test
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    sts_test_examples,
    name="sts-test",
)

Length of Train :  10501
Length of Valid :  1167
Length of Test :  519


In [16]:
# Load model of fine-tuning by NLI
model = SentenceTransformer(nli_model_save_path)

2023/08/08 07:11:53 - Load pretrained SentenceTransformer: output/training_nli_by_Softmaxlossklue-roberta-base-2023-08-08_07-08-05
2023/08/08 07:11:56 - Use pytorch device: cuda


In [17]:
# Use CosineSimilarityLoss
train_loss = losses.CosineSimilarityLoss(model=model)

# warmup steps
warmup_steps = math.ceil(len(sts_train_examples) * sts_num_epochs / train_batch_size * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Trainingㅁ
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=dev_evaluator,
    epochs=sts_num_epochs,
    evaluation_steps=int(len(train_dataloader)*0.1),
    warmup_steps=warmup_steps,
    output_path=sts_model_save_path
)

2023/08/08 07:11:56 - Warmup-steps: 132


Iteration:   9%|▉         | 31/329 [00:05<00:55,  5.33it/s]
                                                           
Epoch:   0%|          | 0/4 [00:05<?, ?it/s]:55,  5.33it/s]

2023/08/08 07:12:02 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset in epoch 0 after 32 steps:



                                                           
Epoch:   0%|          | 0/4 [00:09<?, ?it/s]:55,  5.33it/s]
                                                           
Epoch:   0%|          | 0/4 [00:09<?, ?it/s]:55,  5.33it/s]
                                                           
Epoch:   0%|          | 0/4 [00:09<?, ?it/s]:55,  5.33it/s]
                                                           
Epoch:   0%|          | 0/4 [00:09<?, ?it/s]:55,  5.33it/s]
                                                           
Epoch:   0%|          | 0/4 [00:09<?, ?it/s]:55,  5.33it/s]

2023/08/08 07:12:06 - Cosine-Similarity :	Pearson: 0.8717	Spearman: 0.8755
2023/08/08 07:12:06 - Manhattan-Distance:	Pearson: 0.8418	Spearman: 0.8549
2023/08/08 07:12:06 - Euclidean-Distance:	Pearson: 0.8409	Spearman: 0.8543
2023/08/08 07:12:06 - Dot-Product-Similarity:	Pearson: 0.8312	Spearman: 0.8292
2023/08/08 07:12:06 - Save model to output/training_sts_by_Softmaxlossklue-roberta-base-2023-08-08_07-08-05


...

2023/08/08 07:18:56 - Cosine-Similarity :	Pearson: 0.9637	Spearman: 0.9258
2023/08/08 07:18:56 - Manhattan-Distance:	Pearson: 0.9575	Spearman: 0.9239
2023/08/08 07:18:56 - Euclidean-Distance:	Pearson: 0.9574	Spearman: 0.9235
2023/08/08 07:18:56 - Dot-Product-Similarity:	Pearson: 0.9556	Spearman: 0.9110



Epoch:  75%|███████▌  | 3/4 [07:01<01:46, 106.25s/it]

2023/08/08 07:18:57 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 3:


Epoch: 100%|██████████| 4/4 [07:05<00:00, 106.26s/it]

2023/08/08 07:19:01 - Cosine-Similarity :	Pearson: 0.9637	Spearman: 0.9258
2023/08/08 07:19:01 - Manhattan-Distance:	Pearson: 0.9575	Spearman: 0.9239
2023/08/08 07:19:01 - Euclidean-Distance:	Pearson: 0.9574	Spearman: 0.9235
2023/08/08 07:19:01 - Dot-Product-Similarity:	Pearson: 0.9556	Spearman: 0.9110


In [18]:
# evaluation sts-test
test_evaluator(model, output_path=sts_model_save_path)

2023/08/08 07:19:01 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-test dataset:
2023/08/08 07:19:03 - Cosine-Similarity :	Pearson: 0.8838	Spearman: 0.8855
2023/08/08 07:19:03 - Manhattan-Distance:	Pearson: 0.8849	Spearman: 0.8817
2023/08/08 07:19:03 - Euclidean-Distance:	Pearson: 0.8847	Spearman: 0.8816
2023/08/08 07:19:03 - Dot-Product-Similarity:	Pearson: 0.8698	Spearman: 0.8679


0.8855367274673944

In [10]:
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import paired_cosine_distances, paired_euclidean_distances, paired_manhattan_distances
from scipy.stats import pearsonr

In [20]:
import json
import pandas as pd

dset = []

with open("cbf_track_names.json") as fp:
    for l in fp:
        d = json.loads(l)
        dset.append(d)

print(dset[0])
df = pd.DataFrame(dset)

{'seed_track_nm': 'Seasons (Feat. Harley Bird) (Futuristik & Whogaux Remix)', 'seed_track_artist_nm_list': ['Cadmium', 'Rival'], 'similar_track_nm': 'Seasons (Futuristik & Whogaux Remix)', 'similar_track_artist_nm_list': ['Cadmium', 'Harley Bird', 'Rival'], 'seed_track_nm_rnm': 'seasons', 'similar_track_nm_rnm': 'seasons'}


In [2]:
import numpy as np
import pandas as pd
file1 = pd.read_csv('cbf_track.csv')
file1 = file1.set_index(np.arange(len(file1)))

In [5]:
file1['artist_same'] = 0
for i in range(len(file1)):
    if file1['track_artist'][i] == file1['similar_track_artist'][i]:
        file1['artist_same'][i]=1

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [11]:
file1.to_csv('cbf_track.csv')

In [72]:
file1 = pd.read_parquet('rep_track.parquet')
file1=file1.set_index(np.arange(len(file1)))
file1['merge_key'] = file1['track_nm_notbracspace']+file1['artist_ids']
file1['similar_merge_key'] = file1['similar_track_nm_notbracspace']+file1['similar_artist_ids']

In [7]:
new_df = pd.DataFrame({'sentence1':file1['merge_key'], 'sentence2':file1['similar_merge_key'],'label':0})

In [8]:
sentence1 = []
sentence2 = []
labels = []
for i in range(len(new_df)):
    sentence1.append(new_df['sentence1'][i])
    sentence2.append(new_df['sentence2'][i])
    labels.append(new_df['label'][i])

In [9]:
model = SentenceTransformer(sts_model_save_path)

corpus_embeddings = model.encode(sentence1, convert_to_tensor=True) # senetence1 유사도
query_embeddings = model.encode(sentence2, convert_to_tensor=True) # sentence2 유사도

def cosine_similarity_manual(x, y, small_number=1e-8): # sentence1과 sentence2의 임베딩값으로 유사도 계산
    result =  torch.dot(x, y) / (torch.linalg.norm(x) * torch.linalg.norm(y) + small_number)
    return result

test_scores = []
for i in range(len(sentence1)):
    score = cosine_similarity_manual(corpus_embeddings[i],query_embeddings[i])
    score=score.cpu().detach().numpy()
    test_scores.append(score)

test_scores = np.array(test_scores) # 모델 예측값
y_pred = np.where(test_scores>=0.6, 1, 0) # klue에서 3.0을 기준으로 binary label을 만들었기에, normalize 기준 threshold: 0.6
labels = np.array(labels)
y_label = np.where(labels >= 0.6, 1, 0)

NameError: name 'SentenceTransformer' is not defined

In [76]:
corpus_embeddings = corpus_embeddings.cpu().detach().numpy()
query_embeddings = query_embeddings.cpu().detach().numpy()

cosine_scores = 1 - (paired_cosine_distances(corpus_embeddings, query_embeddings))
manhattan_distances = -paired_manhattan_distances(corpus_embeddings, query_embeddings)
euclidean_distances = -paired_euclidean_distances(corpus_embeddings, query_embeddings)
dot_products = [np.dot(emb1, emb2) for emb1, emb2 in zip(corpus_embeddings, query_embeddings)]

In [77]:
new_df['sentence_bert_notblac_label']= cosine_scores

In [79]:
new_df['sentence_bert_notblac_label'].describe()

count    902099.000000
mean          0.999852
std           0.006305
min           0.046246
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: sentence_bert_notblac_label, dtype: float64

In [71]:
len(new_df[new_df['sentence_bert_notblac_label']<=0.6])

6596

In [80]:
len(new_df[new_df['sentence_bert_notblac_label']<1])

756